In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

In [2]:
# 123-D feature vectors
# 61 phoneme labels mapped to 39 classes
num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 61
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length
dims = 1

In [3]:
FEATURE_FILES = "feature_files"
FEATURE_LABELS = "feature_labels"
TRAIN = "TRAIN"
TEST = "TEST"

def get_data():
    x = np.array([])
    y = np.array([])
    # GET TRAINING DATA
    for file_name in os.listdir(FEATURE_FILES + "/" + TRAIN):
        x_s = np.load(os.path.join(FEATURE_FILES, TRAIN, file_name))
        x = np.vstack((x, x_s)) if x.size else x_s
        break

    for file_name in os.listdir(FEATURE_LABELS + "/" + TRAIN):
        y_s = np.load(os.path.join(FEATURE_LABELS, TRAIN, file_name))
        y = np.vstack((y, y_s)) if y.size else y_s
        break
    
    # x = x.reshape((batch_size, -1, dims))
    return x, y
    
x, y = get_data()

# x.shape should be (batch_size, max_time_step, 26)

In [4]:
def generate_data():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0
    
    print(x.shape)
    print(y.shape)

    x = x.reshape((batch_size, -1, 1))  # 3-D because tf.nn.dynamic_rnn only takes 3-D tensors
    y = y.reshape((batch_size, -1))

    return (x, y)

def generate_sine():
    x = np.linspace(0, 100, total_series_length)
    y = np.cos(x)
    
    x = x.reshape((batch_size, -1, 1))  # 3-D because tf.nn.dynamic_rnn only takes 3-D tensors
    y = y.reshape((batch_size, -1))
    return x, y

x, y = generate_data()

(50000,)
(50000,)


In [5]:
x = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length, dims], name="input_placeholder")
y = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length], name="label_placeholder")

# LSTM
# cell_state = tf.placeholder(tf.float32, [batch_size, state_size])
# hidden_state = tf.placeholder(tf.float32, [batch_size, state_size])
# init_state = tf.contrib.rnn.LSTMStateTuple(cell_state, hidden_state)

# RNN
init_state = tf.placeholder(tf.float32, [batch_size, state_size])


W = tf.Variable(np.random.rand(state_size, num_classes), dtype=tf.float32) # W is 4x2
b = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)# b is 1x2

# Unpack columns
labels_series = tf.unstack(y, axis=1)

# Forward passes

# RNN
cell = tf.contrib.rnn.BasicRNNCell(state_size)

#LSTM
# cell = tf.contrib.rnn.BasicLSTMCell(state_size, state_is_tuple=True)

states_series, current_state = tf.nn.dynamic_rnn(cell=cell, inputs=x, initial_state=init_state)

logits_series = [tf.matmul(state, W) + b for state in tf.unstack(states_series, axis=1)]
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)



In [6]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    loss_list = []

    for epoch_idx in range(num_epochs):
        x1,y1 = generate_sine()
        # RNN
        _current_state = np.zeros((batch_size, state_size))
        
        # LSTM
#         _current_cell_state = np.zeros((batch_size, state_size))
#         _current_hidden_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x1[:,start_idx:end_idx]
            batchY = y1[:,start_idx:end_idx]
            
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    x:batchX,
                    y:batchY,
                    # RNN
                    init_state: _current_state,
                    
                    # LSTM
#                     cell_state: _current_cell_state,
#                     hidden_state: _current_hidden_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
New data, epoch 0
Step 0 Loss 3.92492
Step 100 Loss 0.0108291
Step 200 Loss 0.00532531
Step 300 Loss 0.00354094
Step 400 Loss 0.00265346
Step 500 Loss 0.00212172
Step 600 Loss 0.00176774
New data, epoch 1
Step 0 Loss 0.0972658
Step 100 Loss 0.00140235
Step 200 Loss 0.00122597
Step 300 Loss 0.00109758
Step 400 Loss 0.000994783
Step 500 Loss 0.000909034
Step 600 Loss 0.000837095
New data, epoch 2
Step 0 Loss 0.101029
Step 100 Loss 0.000752752
Step 200 Loss 0.000695321
Step 300 Loss 0.000651719
Step 400 Loss 0.00061374
Step 500 Loss 0.000580023
Step 600 Loss 0.00055
New data, epoch 3
Step 0 Loss 0.102703
Step 100 Loss 0.000515659
Step 200 Loss 0.000486588
Step 300 Loss 0.000464284


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda2/envs/rnn/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/anaconda2/envs/rnn/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/anaconda2/envs/rnn/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/anaconda2/envs/rnn/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/anaconda2/envs/rnn/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/anaconda2/envs/rnn/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/anaconda2/envs/

IndexError: string index out of range